

## Challenge of processing large amounts of data
<ul>
<li>How to process is quickly?
<li>So how do we go about making the problem map so that it can be distributed computation?
<li> Distributed/Parrallel Programming is hard
</ul>

##Mapreduce addresses all the challengs
<ul>
<li>Google's computational/data manipulation model
<li> Elegant way to work with big data
</ul>

## Map Reduce and the New Software Stack
<ul>
<li> Covered in Chapter 2 in the Ullman book
<li> Python support packages https://pypi.python.org/pypi/mrjob
<li> map() and reduce() are basic built in functions in python https://docs.python.org/2/library/functions.html
</ul>


## Built-in Python Functional Programming Tools
https://docs.python.org/2/tutorial/datastructures.html

### Python Map Function
For each value in a sequence, process each one and output a new result for each element.

In [6]:
def cube(x): return x*x*x

map(cube,range(1,11))

[1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]

If there are more parameters then each could be an array, and they are applied together one element at a time

In [7]:
seq = range(8)
def add(x,y): return x+y
map(add, seq,seq)

[0, 2, 4, 6, 8, 10, 12, 14]

## Python Reduce Function
For an array passed in, compute a single result.
So a reduce function always has two paramters to carry the result foward with the next element in the sequence.
The operation starts by using the first two values then passes the result with the next element to the function...

In [8]:
result = map(add, seq,seq)
reduce(add, result)  # adding each element of the result together

56

If there is only one value in a sequence then that element is returned; if the sequence is empty, an exception is raised.

##Word Count using python in a map reduce manner
Using the Canterbury Corpus Test file from http://compression.ca/act/files/canterbury.zip,
We will attempt to count the each unique word.


In [40]:
import re
import pandas as pd
import numpy as np
aliceFile = open('data/canterbury/alice29.txt','r')
map1=[]
# Create the map of words with prelminary counts
for line in aliceFile:
    words = line.split()
    for word in words:
        subwords = re.sub("[\W\d]"," ",word.strip())
        for w in subwords.split():
            map1.append([w,1])

#sort the map            
map2 = sorted(map1)

#Separate the map into groups by the key values
df = pd.DataFrame(map2)

uniquewords = df[0].unique()
DataFrameDict = {elem : pd.DataFrame for elem in uniquewords}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df[:][df[0] == key]


In [41]:
def wordcount(x,y):
    x[1] = x[1] + y[1]
    return x

#Add up the counts using reduce

for uw in uniquewords:
    uarray = np.array(DataFrameDict[uw])
    print reduce(wordcount,uarray)


['A' 17L]
['ADVENTURES' 1L]
['ALICE' 3L]
['ALL' 4L]
['AND' 3L]
['ARE' 6L]
['AT' 1L]
['Ada' 1L]
['Adventures' 2L]
['Advice' 1L]
['After' 6L]
['Ah' 5L]
['Ahem' 1L]
['Alas' 1L]
['Alice' 395L]
['All' 5L]
['Allow' 1L]
['Always' 1L]
['Ambition' 1L]
['An' 5L]
['And' 67L]
['Ann' 4L]
['Antipathies' 1L]
['Anything' 1L]
['Are' 4L]
['Arithmetic' 1L]
['As' 17L]
['At' 9L]
['Atheling' 1L]
['Australia' 1L]
['BE' 1L]
['BEE' 1L]
['BEFORE' 1L]
['BEG' 1L]
['BEST' 2L]
['BOOTS' 1L]
['BUSY' 1L]
['Back' 1L]
['Be' 2L]
['Beau' 4L]
['Beautiful' 5L]
['Because' 1L]
['Before' 1L]
['Begin' 1L]
['Behead' 1L]
['Besides' 1L]
['Between' 1L]
['Bill' 16L]
['Birds' 1L]
['Boots' 1L]
['Brandy' 1L]
['Bring' 1L]
['But' 37L]
['By' 4L]
['C' 1L]
['CAN' 4L]
['CHAPTER' 12L]
['CHORUS' 2L]
['COULD' 4L]
['COURT' 1L]
['CURTSEYING' 1L]
['Call' 4L]
['Can' 2L]
['Canary' 1L]
['Canterbury' 1L]
['Carroll' 1L]
['Cat' 26L]
['Catch' 1L]
['Caterpillar' 27L]
['Caucus' 3L]
['Certainly' 1L]
['Cheshire' 7L]
['Chorus' 1L]
['Christmas' 1L]
['Classics'